In [10]:
import csv
import datetime
import re
from datetime import timedelta

def time_to_srt_format(time):
    return str(datetime.timedelta(seconds=time)).split('.')[0]

def time_difference(time_str):
    start, end = time_str.split(' - ')
    try:
        FMT = "%H:%M:%S.%f"
        tdelta = datetime.datetime.strptime(end, FMT) - datetime.datetime.strptime(start, FMT)
    except ValueError:
        try:
            FMT = "%H:%M:%S"
            tdelta = datetime.datetime.strptime(end, FMT) - datetime.datetime.strptime(start, FMT)
        except ValueError:
            return 0
    return tdelta.total_seconds()

def clean_text(text):
    # Remove special characters from the text
    cleaned_text = re.sub(r"[^\w\s]", "", text)
    return cleaned_text


csv_file="../"

with open(csv_file, 'r') as csv_file, open('../subtitle.srt', 'w') as srt_file:
    csv_reader = csv.reader(csv_file)
    next(csv_reader)  # Skip header
    counter = 1
    last_time = 0
    for row in csv_reader:
        srt_file.write(str(counter) + '\n')
        if row[3]:  # If Timecode Range is not empty
            start_time, end_time = row[3].split(' - ')
            time_diff = time_difference(row[3])
            start_seconds = last_time
            end_seconds = last_time + time_diff
            last_time = end_seconds
            start_time = time_to_srt_format(start_seconds)
            end_time = time_to_srt_format(end_seconds)
            srt_file.write(start_time + ',000 --> ' + end_time + ',000\n')
        else:
            start_time = time_to_srt_format(last_time)  # Start time based on last saved end time
            last_time += 5  # Add 5 seconds to the last known time
            end_time = time_to_srt_format(last_time)
            srt_file.write(start_time + ',000 --> ' + end_time + ',000\n')
        srt_file.write(clean_text(row[0]) + '\n\n')  # Cleaned text
        counter += 1
